In [1]:
from utils.getter import *
import torch.utils.data as data
import torch
import torchvision.models as models
from tqdm import tqdm
import torch.nn as nn

[nltk_data] Downloading package stopwords to C:\Users\Kay
[nltk_data]     Kieran\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
transforms = Compose([
    Resize((300,300)),
    ToTensor(),
    Normalize(),
])

In [3]:
data_path = "datasets/datasets/Garbage Classification"
voc_path = "datasets/datasets/VOC/images"
voc_anno = "datasets/datasets/VOC/annotations/pascal_train2012.json"
trainset = ImageClassificationDataset(data_path+ "/train", transforms= transforms, shuffle=True)
valset = ImageClassificationDataset(data_path+ "/val", transforms= transforms, shuffle=True)

print(trainset)
print(valset)

Custom Dataset for Image Classification
-------------------------------
Number of samples: 2271
Number of classes: 6

Custom Dataset for Image Classification
-------------------------------
Number of samples: 256
Number of classes: 6



In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using", device)

Using cuda


In [5]:
# Dataloader
BATCH_SIZE = 32
trainloader = data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True)
valloader = data.DataLoader(valset, batch_size=BATCH_SIZE, shuffle=True)

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam
metrics = [AccuracyMetric(decimals=3)]

In [7]:
NUM_CLASSES = len(trainset.classes)
model = ResNet34(NUM_CLASSES,
                 lr = 1e-4,
                 criterion= criterion, 
                 optimizer= optimizer,
                 metrics=  metrics,
                 device = device)
#load_checkpoint(model, "weights/ResNet34-12.pth")

In [8]:
for batch in trainloader:
    model.optimizer.zero_grad()
    img = batch['img'].to(device)
    label = batch['label'].to(device)
    outputs = model(img)
    loss = criterion(outputs, label)
    print(loss.item())
    #loss.backward()
    break

2.1207523345947266


In [ ]:
cp = Checkpoint(save_per_epoch=6)
trainer = Trainer(model,
                 trainloader, 
                 valloader,
                 checkpoint = cp, 
                 evaluate_per_epoch = 2)

trainer.fit(num_epochs=30)